# Experiment 001: Baseline with Pre-optimized CSV

Following the seed prompt strategy:
1. Load pre-optimized submission (sample_submission.csv as starting point)
2. Score the baseline
3. Apply fix_direction rotation tightening
4. Validate no overlaps
5. Generate submission

In [1]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
import warnings
warnings.filterwarnings('ignore')

getcontext().prec = 30
scale_factor = 1

print('Libraries loaded successfully')

Libraries loaded successfully


In [2]:
# Christmas Tree class from the kernels
class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                (float(Decimal('0.0') * scale_factor), float(tip_y * scale_factor)),
                (float(top_w / Decimal('2') * scale_factor), float(tier_1_y * scale_factor)),
                (float(top_w / Decimal('4') * scale_factor), float(tier_1_y * scale_factor)),
                (float(mid_w / Decimal('2') * scale_factor), float(tier_2_y * scale_factor)),
                (float(mid_w / Decimal('4') * scale_factor), float(tier_2_y * scale_factor)),
                (float(base_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
                (float(trunk_w / Decimal('2') * scale_factor), float(base_y * scale_factor)),
                (float(trunk_w / Decimal('2') * scale_factor), float(trunk_bottom_y * scale_factor)),
                (float(-(trunk_w / Decimal('2')) * scale_factor), float(trunk_bottom_y * scale_factor)),
                (float(-(trunk_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
                (float(-(base_w / Decimal('2')) * scale_factor), float(base_y * scale_factor)),
                (float(-(mid_w / Decimal('4')) * scale_factor), float(tier_2_y * scale_factor)),
                (float(-(mid_w / Decimal('2')) * scale_factor), float(tier_2_y * scale_factor)),
                (float(-(top_w / Decimal('4')) * scale_factor), float(tier_1_y * scale_factor)),
                (float(-(top_w / Decimal('2')) * scale_factor), float(tier_1_y * scale_factor)),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

    def clone(self):
        return ChristmasTree(
            center_x=str(self.center_x),
            center_y=str(self.center_y),
            angle=str(self.angle),
        )

print('ChristmasTree class defined')

ChristmasTree class defined


In [3]:
# Helper functions
def load_configuration_from_df(n, df):
    """Loads all trees for a given N from the submission DataFrame."""
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"])[1:]  # Remove 's' prefix
        y = str(row["y"])[1:]
        deg = str(row["deg"])[1:]
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees

def get_score(trees, n):
    """Calculates the score (S^2 / N) for a given configuration of trees."""
    if not trees:
        return 0.0
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / float(scale_factor) for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    side_length = max(max_x - min_x, max_y - min_y)
    return side_length**2 / n

def has_overlap(trees):
    """Check if any two ChristmasTree polygons overlap."""
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def score_and_validate_submission(file_path, max_n=200):
    """Reads a submission CSV, calculates the total score, and checks for overlaps."""
    df = pd.read_csv(file_path)
    total_score = 0.0
    failed_overlap_n = []
    
    for n in range(1, max_n + 1):
        trees = load_configuration_from_df(n, df)
        if trees:
            current_score = get_score(trees, n)
            total_score += current_score
            if has_overlap(trees):
                failed_overlap_n.append(n)
    
    return {
        "status": "SUCCESS" if not failed_overlap_n else "FAILED (Overlaps)",
        "total_score": total_score,
        "failed_overlap_n": failed_overlap_n
    }

print('Helper functions defined')

Helper functions defined


In [4]:
# Load and score the sample submission (baseline)
baseline_path = '/home/data/sample_submission.csv'
print(f'Loading baseline from: {baseline_path}')

result = score_and_validate_submission(baseline_path)
print(f"\nBaseline Score: {result['total_score']:.6f}")
print(f"Status: {result['status']}")
if result['failed_overlap_n']:
    print(f"Overlaps in N: {result['failed_overlap_n'][:10]}...")

Loading baseline from: /home/data/sample_submission.csv



Baseline Score: 173.652299
Status: SUCCESS


In [ ]:
# fix_direction rotation tightening functions
def calculate_bbox_side_at_angle(angle_deg, points):
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix_T = np.array([[c, s], [-s, c]])
    rotated_points = points.dot(rot_matrix_T)
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation(trees):
    """Find optimal rotation angle to minimize bounding box."""
    all_points = []
    for tree in trees:
        all_points.extend(list(tree.polygon.exterior.coords))
    points_np = np.array(all_points)
    
    try:
        hull_points = points_np[ConvexHull(points_np).vertices]
    except:
        return Decimal(0), 0.0
    
    initial_side = calculate_bbox_side_at_angle(0, hull_points)
    
    res = minimize_scalar(lambda a: calculate_bbox_side_at_angle(a, hull_points),
                          bounds=(0.001, 89.999), method='bounded')
    found_angle_deg = res.x
    found_side = res.fun
    
    improvement = initial_side - found_side
    EPSILON = 1e-8
    
    if improvement > EPSILON:
        return Decimal(found_side) / scale_factor, found_angle_deg
    else:
        return Decimal(initial_side) / scale_factor, 0.0

def apply_rotation(trees, angle_deg):
    """Apply rotation to all trees around their collective center."""
    if not trees or abs(angle_deg) < 1e-9:
        return [t.clone() for t in trees]
    
    bounds = [t.polygon.bounds for t in trees]
    min_x = min(b[0] for b in bounds)
    min_y = min(b[1] for b in bounds)
    max_x = max(b[2] for b in bounds)
    max_y = max(b[3] for b in bounds)
    rotation_center = np.array([(min_x + max_x) / 2.0, (min_y + max_y) / 2.0])
    
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, -s], [s, c]])
    
    points = np.array([[float(t.center_x), float(t.center_y)] for t in trees])
    shifted = points - rotation_center
    rotated = shifted.dot(rot_matrix.T) + rotation_center
    
    rotated_trees = []
    for i in range(len(trees)):
        new_tree = ChristmasTree(
            str(rotated[i, 0]), 
            str(rotated[i, 1]),
            str(float(trees[i].angle) + angle_deg)
        )
        rotated_trees.append(new_tree)
    return rotated_trees

print('Rotation optimization functions defined')

In [ ]:
# Apply fix_direction to all N configurations
import copy

df = pd.read_csv(baseline_path)

# Store optimized configurations
optimized_configs = {}
improved_count = 0
total_improvement = 0.0

print('Applying fix_direction rotation tightening...')
for n in range(1, 201):
    trees = load_configuration_from_df(n, df)
    if not trees:
        continue
    
    original_score = get_score(trees, n)
    best_side, best_angle = optimize_rotation(trees)
    
    if abs(best_angle) > 1e-6:
        rotated_trees = apply_rotation(trees, best_angle)
        new_score = get_score(rotated_trees, n)
        
        if new_score < original_score:
            optimized_configs[n] = rotated_trees
            improvement = original_score - new_score
            total_improvement += improvement
            improved_count += 1
            if n <= 10 or n % 50 == 0:
                print(f'  N={n}: {original_score:.6f} -> {new_score:.6f} (angle={best_angle:.2f}°)')
        else:
            optimized_configs[n] = trees
    else:
        optimized_configs[n] = trees

print(f'\nImproved {improved_count} configurations')
print(f'Total improvement: {total_improvement:.6f}')

In [ ]:
# Generate new submission with optimized configurations
def generate_submission(optimized_configs, output_path):
    """Generate submission CSV from optimized configurations."""
    rows = []
    for n in range(1, 201):
        trees = optimized_configs.get(n, [])
        for i, tree in enumerate(trees):
            row = {
                'id': f'{n:03d}_{i}',
                'x': f's{float(tree.center_x)}',
                'y': f's{float(tree.center_y)}',
                'deg': f's{float(tree.angle)}'
            }
            rows.append(row)
    
    df_out = pd.DataFrame(rows)
    df_out.to_csv(output_path, index=False)
    return df_out

submission_path = '/home/submission/submission.csv'
df_out = generate_submission(optimized_configs, submission_path)
print(f'Generated submission with {len(df_out)} rows')
print(f'Saved to: {submission_path}')

In [ ]:
# Validate and score the new submission
print('\nValidating new submission...')
final_result = score_and_validate_submission(submission_path)

print(f"\nFinal Score: {final_result['total_score']:.6f}")
print(f"Status: {final_result['status']}")
if final_result['failed_overlap_n']:
    print(f"Overlaps in N: {final_result['failed_overlap_n']}")

# Compare with baseline
print(f"\nBaseline Score: {result['total_score']:.6f}")
print(f"New Score: {final_result['total_score']:.6f}")
print(f"Improvement: {result['total_score'] - final_result['total_score']:.6f}")

In [ ]:
# Check submission format
df_check = pd.read_csv(submission_path)
print('Submission format check:')
print(df_check.head(10))
print(f'\nTotal rows: {len(df_check)}')
print(f'Expected rows: {sum(range(1, 201))} = 20100')